# Домашнее задание 3
## Сравнение интересов аудитории телеканалов НТВ и Дождь с помощью тематического моделирования LDA

Задача:
Сравнить интересы аудитории телеканолов НТВ и Дождь с помощью методов тематического моделирования
1. Получить данные по аудитории из социальной сети ВК
2. Зарегистрировать приложение, получить app_id, access_token
3. Скачать данные по пользователям в каждой из групп (id групп ВК даны ниже, tvrain_id, ntv_id)
4. Взять небольшую выборку из каждой совокупности телезрителей(около 1000-2000 человек, т.к. 300k-400k слишком много), с которыми работать дальше
5. Обучить LDA модель на их подписках
6. По группам, на которые подписаны эти люди, полуичть ключевые слова групп, на которые они подписаны
7. Получить распределение интересов людей для каждой совокупности, сравнить на графике

In [59]:
import seaborn as sb
import sys  
import pandas as pd
import requests
import numpy as np
import datetime
import time
import pickle
import matplotlib.pyplot as plt
import time
%matplotlib inline

from bs4 import BeautifulSoup as bs
import urllib
from tqdm import tqdm_notebook as tqdm

In [3]:
def load_obj(name):
    with open(name + '.pkl', 'rb') as f:
        return pickle.load(f)

def save_obj(obj, name):
    with open(name + '.pkl', 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)

#### Для использования VK API необходимо создать приложение в VK

1. Создать приложение по адресу https://vk.com/apps?act=manage (кнопка "создать приложение")
2. При создании указать название, описание (можно любые), категория  - прочее. Тип - standalone приложение
3. В настройках получить **app_id**. App_id потребуется для получения access token
4. Авторизовать пользователя (получить access token) можно по адресу: https://vk.com/dev/first_guide, в правилах нас интересует пункт 3 **Авторизация пользователя**
5. После того, как ознакомитесь с авторизацией пользователя, скопируйте в адресную строку такой запрос https://oauth.vk.com/authorize?client_id=5490057&display=page&redirect_uri=https://oauth.vk.com/blank.html&scope=friends&response_type=token&v=5.52, где число **5490057** замените на число, которое получите для вашего **app_id**
6. Нажмите Enter. Откроется окно с запросом прав. В нем отображаются название приложения, иконки прав доступа, и Ваши имя с фамилией. Нажмите «Разрешить». Вы попадете на новую страницу с предупреждением о том, что токен нельзя копировать и передавать третьим лицам. В адресной строке будет URL https://oauth.vk.com/blank.html, а после # Вы увидите дополнительные параметры — access_token, expires_in и user_id. Токен может выглядеть, например, так: 51eff86578a3bbbcb5c7043a122a69fd04dca057ac821dd7afd7c2d8e35b60172d45a26599c08034cc40a
7. Токен — это Ваш ключ доступа. При выполнении определенных условий человек, получивший Ваш токен, может нанести существенный ущерб Вашим данным и данным других людей. Поэтому очень важно не передавать свой токен третьим лицам
8. Поле expires_in содержит время жизни токена в секундах. 86400 секунд — это ровно сутки. Через сутки полученный токен перестанет действовать, для продолжения работы нужно будет получить новый по такому же алгоритму

In [4]:
# use your own app id and respective tokens

# скопируйте сюда ваши app_id и access_token, полученные по методу, описанному выше
app_id = '6985218'
access_token = '3aab1b49c1a9f9bf99085e2c31a47f4e211115861d282ab4fec83afa9e8feee9dbf8a7da550b4868183d4'

# id групп ВК Дождя и НТВ
tvrain_id = 17568841
ntv_id = 28658784

In [5]:
# проверка работы API и авторизации пользователя. Если возникает ошибка, то, возможно, access token необходимо обновить
check_id = 1

# api call and test
def vk_get_response(method, parameters, token):
    url = 'https://api.vk.com/method/' + method + '?' + parameters + '&access_token=' + token
#     print url
    return(requests.get(url).json())

answer = vk_get_response(
    'users.get', 'user_ids={0}&v=4.9&lang=ru'.format(check_id), access_token
)['response']
print(answer[0]['first_name'], answer[0]['last_name'])

Павел Дуров


### Получение подписчиков телеканалов НТВ и Дождь в VK

Получим объекты, которые содержат всю информацию о подпиичиках соответствующих групп (указанных в domains) и сохраним их на диск. Получим в итоге два файла - **ntv_subs** и **tvrain_subs** в формате **.pkl** - питоновский формат хранения данных

In [ ]:
domains = ['ntv', 'tvrain']


for group_domain in domains:
    offset = 0
    group_id = group_domain
    fields = """sex,bdate,city,country,home_town,lists,domain,has_mobile,
    contacts,connections,education,universities,followers_count,occupation,last_seen,relation"""
    first_sample = vk_get_response(
        'groups.getMembers', 'group_id={0}&offset={1}&fields={2}&v=4.9&lang=ru'.format(
            group_id, offset, fields
        ), token=access_token
    )
    community_count = first_sample['response']['count']
    community_members = []
    for i in range(community_count // 1000 + 1):
        offset = i * 1000
        try:
            answer = vk_get_response(
                'groups.getMembers', 'group_id={0}&offset={1}&fields={2}&v=4.9&lang=ru'.format(
                    group_id, offset, fields), token=access_token
            )
            print("Offset: ", offset)
        except:
            print("Offset: ", offset, " Error")
        community_members += answer['response']['users']
    save_obj(community_members, '{}_subs'.format(group_domain))

In [7]:
community_ntv = load_obj('ntv_subs')
community_tvrain = load_obj('tvrain_subs')

In [8]:
community_ntv_df = pd.DataFrame(community_ntv)
community_tvrain_df = pd.DataFrame(community_tvrain)

Посмотрим, что загрузилось

In [9]:
print(len(community_ntv_df))
print(len(community_tvrain_df))

375491
444925


In [10]:
community_ntv_df.head(5)

,bdate,city,country,deactivated,domain,education_form,education_status,facebook,facebook_name,faculty,...,occupation,relation,relation_partner,sex,skype,twitter,uid,universities,university,university_name
0,10.2,2.0,1.0,NaN,rozenkat,NaN,NaN,NaN,NaN,NaN,...,"{'type': 'university', 'id': 1, 'name': 'СПбГУ'}",NaN,NaN,1,NaN,NaN,279,NaN,NaN,NaN
1,NaN,NaN,NaN,banned,id348,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,1,NaN,NaN,348,NaN,NaN,NaN
2,NaN,0.0,0.0,NaN,id510,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,2,NaN,NaN,510,NaN,NaN,NaN
3,NaN,NaN,NaN,banned,id619,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,2,NaN,NaN,619,NaN,NaN,NaN
4,20.5.1987,0.0,0.0,NaN,ilyaa.orlov,NaN,NaN,NaN,NaN,0.0,...,NaN,5.0,NaN,2,NaN,NaN,914,[],0.0,


интересный признак deactivated

In [13]:
community_ntv_df['deactivated'].value_counts()

deleted    35492
banned     12794
Name: deactivated, dtype: int64

Оставляем строки только с пустым атрибутом deactivated

In [18]:
community_ntv_df = community_ntv_df.loc[community_ntv_df['deactivated'].isna()]
community_tvrain_df = community_tvrain_df.loc[community_tvrain_df['deactivated'].isna()]

In [19]:
print(len(community_ntv_df))
print(len(community_tvrain_df))

327205
375419


Сначала получим всех уникальных подписчиков НТВ и Дождя с помощью unique. Далее с помощью numpy.random необходимо выбрать небольшой sample (например, по 1000 из каждой группы) таких людей и объединить их вместе

In [25]:
ntv_uids = community_ntv_df.uid.unique().tolist()
tvrain_uids = community_tvrain_df.uid.unique().tolist()

In [30]:
# получить общий список людей из двух выборок НТВ и Дождя, всего должно быть в итоге около 2000 человек
uids = np.concatenate([np.random.choice(ntv_uids, size=1000), np.random.choice(tvrain_uids, size=1000)])

In [31]:
uids = np.unique(uids)
len(uids)

1995

In [33]:
# получить подписки этих пользователей
print_counter = 0
final_data = []

for uid in uids:
    try:
        user_subs = vk_get_response(
            'users.getSubscriptions', 'user_id={0}&v=4.9&lang=ru'.format(int(uid)), access_token
        )
        time.sleep(0.3)
        final_data.append(user_subs)
    except:
        print("Error")
    if print_counter % 100 == 0:
        print("{0} profiles done".format(print_counter))
    print_counter += 1

0 profiles done
100 profiles done
200 profiles done
300 profiles done
400 profiles done
500 profiles done
600 profiles done
700 profiles done
800 profiles done
900 profiles done
1000 profiles done
1100 profiles done
1200 profiles done
1300 profiles done
1400 profiles done
1500 profiles done
1600 profiles done
1700 profiles done
1800 profiles done
1900 profiles done


In [34]:
final_data[:1]

[{'response': {'users': {'count': 2, 'items': [321, 129244038]},
   'groups': {'count': 17,
    'items': [63731512,
     48944668,
     1441,
     17568841,
     20225241,
     23391543,
     24098496,
     25153764,
     26270763,
     29906641,
     36456996,
     36541026,
     36592958,
     41813928,
     67888368,
     72614618,
     106838393]}}}]

In [35]:
subs_list = []
groups_freq_dict = {}
top_n = 5

for record, uid in zip(final_data, uids):
    try:
        user_subs = record
        if not user_subs.get('response'):
            user_subs = vk_get_response(
                'users.getSubscriptions', 'user_id={0}&v=4.9&lang=ru'.format(int(uid)), access_token
            )
        subs_pd = pd.DataFrame(
            [
                {
                    'groups_count': user_subs['response']['groups'].get('count'),
                    'groups_list': user_subs['response']['groups'].get('items'),
                    'follows_count':user_subs['response']['users'].get('count'),
                    'follows_list': user_subs['response']['users'].get('items'),
                }
            ]
        )

        for group_id in user_subs['response']['groups'].get('items')[:top_n]:
            if groups_freq_dict.get(group_id):
                groups_freq_dict[group_id] += 1
            else:
                groups_freq_dict[group_id] = 1

        subs_pd['subs_count'] = subs_pd['groups_count'] + subs_pd['follows_count']
        subs_list.append(subs_pd)
    except:
#         print(user_subs)
        pass
    if len(subs_list) % 100 == 0:
        print("Processed {0} users".format(len(subs_list)))

Processed 100 users
Processed 200 users
Processed 300 users
Processed 300 users
Processed 400 users
Processed 500 users
Processed 600 users
Processed 600 users
Processed 700 users
Processed 800 users
Processed 900 users
Processed 900 users
Processed 900 users
Processed 1000 users
Processed 1100 users
Processed 1200 users
Processed 1300 users
Processed 1400 users
Processed 1500 users
Processed 1600 users


Самые популярные группы

In [36]:
sorted([(key, val) for key, val in groups_freq_dict.items()], key=lambda x: x[1], reverse=True)[:5]

[(17568841, 284),
 (28658784, 162),
 (15755094, 60),
 (29534144, 41),
 (2158488, 36)]

Загрузка постов со стен групп

In [47]:
group_doc_dict = {}
counter = 0
groups_freq_dict_top5 = groups_freq_dict

for group_id, freq in groups_freq_dict_top5.items():
    counter += 1
    try:
        check = vk_get_response(
            'wall.get',
            'owner_id={0}&count=100&fields=post_type,marked_as_ads&&v=4.9&lang=ru'.format(int(group_id) * -1),
            access_token
        )
        check = check['response']
        group_doc = ''
        if check[0] != 0:
            for post in check[1:]:
                if post.get('marked_as_ads') != 1:
                    group_doc += post['text']
        group_doc_dict[group_id] = group_doc
    except:
        print("Response error. Group id {0}".format(group_id))
        print(check)
    if counter % 100 == 0:
        print("{0} groups extracted".format(counter))
    time.sleep(0.3)

100 groups extracted
200 groups extracted
300 groups extracted
400 groups extracted
500 groups extracted
600 groups extracted
700 groups extracted
800 groups extracted
900 groups extracted
1000 groups extracted
1100 groups extracted
1200 groups extracted
1300 groups extracted
1400 groups extracted
1500 groups extracted
1600 groups extracted
1700 groups extracted
1800 groups extracted
1900 groups extracted
2000 groups extracted
2100 groups extracted
2200 groups extracted
2300 groups extracted
2400 groups extracted
2500 groups extracted
2600 groups extracted
2700 groups extracted
2800 groups extracted
2900 groups extracted
3000 groups extracted
3100 groups extracted
3200 groups extracted
3300 groups extracted
Response error. Group id 41589556
{'error': {'error_code': 15, 'error_msg': 'Access denied: this wall available only for community members', 'request_params': [{'key': 'oauth', 'value': '1'}, {'key': 'method', 'value': 'wall.get'}, {'key': 'owner_id', 'value': '-41589556'}, {'key': 

In [48]:
# сохранить сырые данные по постам групп на диск
save_obj(group_doc_dict, 'group_doc_dict')

In [57]:
from bs4 import BeautifulSoup
import re
import string
from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
import nltk
nltk.download('punkt')
from pymorphy2 import MorphAnalyzer

[nltk_data] Downloading package punkt to /home/vlad/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Конечно, по хорошему этот кусок надо переписать, как нас учили по NLP, но раз уж работает - пусть остается.

In [53]:
chrs_to_delete = string.punctuation + u'»' + u'«' + u'—' + u'“' + u'„' + u'•' + u'#'
translation_table = {ord(c): None for c in chrs_to_delete if c != u'-'}
units = MorphAnalyzer.DEFAULT_UNITS
morph = MorphAnalyzer(result_type=None, units=units)
PortSt = PorterStemmer()
stopw = set(
    [w for w in stopwords.words(['russian', 'english'])]
    + [u'это', u'году', u'года', u'также', u'етот',
       u'которые', u'который', u'которая', u'поэтому',
       u'весь', u'свой', u'мочь', u'eтот', u'например',
       u'какой-то', u'кто-то', u'самый', u'очень', u'несколько',
       u'источник', u'стать', u'время', u'пока', u'однако',
       u'около', u'немного', u'кроме', u'гораздо', u'каждый',
       u'первый', u'вполне', u'из-за', u'из-под',
       u'второй', u'нужно', u'нужный', u'просто', u'большой',
       u'хороший', u'хотеть', u'начать', u'должный', u'новый', u'день',
       u'метр', u'получить', u'далее', u'именно', u'апрель',
       u'сообщать', u'разный', u'говорить', u'делать',
       u'появиться', u'2016',
       u'2015', u'получить', u'иметь', u'составить', u'дать', u'читать',
       u'ничто', u'достаточно', u'использовать',
       u'принять', u'практически',
       u'находиться', u'месяц', u'достаточно', u'что-то', u'часто',
       u'хотеть', u'начаться', u'делать', u'событие', u'составлять',
       u'остаться', u'заявить', u'сделать', u'дело',
       u'примерно', u'попасть', u'хотя', u'лишь', u'первое',
       u'больший', u'решить', u'число', u'идти', u'давать', u'вопрос',
       u'сегодня', u'часть', u'высокий', u'главный', u'случай', u'место',
       u'конец', u'работать', u'работа', u'слово', u'важный', u'сказать']
)

In [54]:
url_start = 'http[s]?://'
url_end = (
    '(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
)
pattern = url_start + url_end

Обработка слов постов групп - трансформация в "хороший" вид. Нормализация и стэмминг, удаление стоп-слов

In [60]:
group_clean_doc_dict = {}
#counter = 0

for group_id, doc in tqdm(group_doc_dict.items()):
    soup = BeautifulSoup(doc, 'html.parser')
    body = ' '.join(
        [tag.string.replace('\\n', ' ').replace('\\r', ' ')
         for tag in soup.descendants if tag.string]
    )
    body = re.sub('\[.*?\]','', body)
    body = re.sub(pattern,'', body)
    if body != '':
        body_clean = body.translate(translation_table).lower().strip()
        words = word_tokenize(body_clean)
        tokens = []
        # stemming and text normalization
        for word in words:
            if re.match('^[a-z0-9-]+$', word) is not None:
                tokens.append(PortSt.stem(word))
            elif word.count('-') > 1:
                tokens.append(word)
            else:
                normal_forms = morph.normal_forms(word)
                tokens.append(normal_forms[0] if normal_forms else word)
        # remove stopwords and leave unique words only
        tokens = filter(
            lambda token: token not in stopw, sorted(set(tokens))
        )

        # remove all words with more than 3 chars
        tokens = filter(lambda token: len(token) > 3, tokens)
    else:
        tokens = []
    #counter += 1
    #if counter % 100 == 0:
        #print("{0} docs processed".format(counter))
    group_clean_doc_dict[group_id] = tokens

group_clean_doc_dict = {key: list(val) for key, val in group_clean_doc_dict.items()}

In [61]:
# сохранить обработанные данные на диск
save_obj(group_clean_doc_dict, 'group_doc_dict_clean')

### Обучение LDA модели

In [62]:
from gensim.corpora import TextCorpus
from gensim.models.ldamulticore import LdaMulticore
from gensim.models.ldamodel import LdaModel

class ListTextCorpus(TextCorpus):

    def get_texts(self):
        for doc in self.input:
            yield doc
                
mycorp = ListTextCorpus(input=group_clean_doc_dict.values())
justlda = LdaModel(
    corpus=mycorp, num_topics=20, passes=12
)

In [63]:
print('LdaModel performance')
for i in range(20):
    terms = justlda.get_topic_terms(i)
    print(i, ' '.join(map(lambda x: mycorp.dictionary.get(x[0]), terms)))

LdaModel performance
0 нeoжидaннo гepoь poбuн пpuнять oкaзaлиcь пpecтyплeние мeлoдpaмa гepou oxoт нaчaльнuкoм
1 жизнь стоить город человек вместе полный спасибо смочь ждать знать
2 -согласный сказанохорошо ☝🏻от ☝🏻ный душисогласный -хорошо душихорошо -истинаот жизнь любить
3 нoминация cмoтрить oпубликoвать пpoживaние пoбeждaть гpaбитель психологуpoвень динoзaвpамион типмна лaпкипроблема
4 skool laurel gumкед 🍦😍кед 🍊кед бyтилиpoвaннoть угломер кpaн-нacoc мультитул пескоструйный
5 масло ингредиент приготовление соль овощ растительный рецепт ложка перец нарезать
6 безмеpный пpоизводитель нuawei мaгазин яблопользователь яблоkый благодаpность камеp таkий уделать
7 convert firebird camino 1979oldsmobil impala hyundaizubastik oфopмлeнный резвиться лoджия комнaт
8 чтoб ecть тoлькo мeнить кoгдa тeбить знaть былo дeнь этoт
9 такогo нayчить подaрoк игноpиpoвать дoрогойя нaчинаeм зaгорелacь возврaтa чье-тo бeзразличие
10 человек друг ждать писать знать группа новое фото жизнь найти
11 який буде тіл

In [64]:
dummy_dict = {key: 0 for key in range(20)}

group_topics_dict_20 = {
    group_id: dict(list(dummy_dict.items()) + justlda.get_document_topics(mycorp.dictionary.doc2bow(text)))
    for group_id, text in group_clean_doc_dict.items()
}
check_pd_20 = pd.DataFrame.from_dict(group_topics_dict_20, orient='index')
check_pd_20.head(10)
print("Group distribution by the most relevant topic")
pd.Series.round(check_pd_20.idxmax(axis=1).value_counts() * 1. / len(check_pd_20), 2)

Group distribution by the most relevant topic


10    0.52
13    0.29
14    0.11
5     0.04
0     0.02
8     0.01
16    0.01
11    0.01
6     0.00
15    0.00
3     0.00
18    0.00
2     0.00
4     0.00
dtype: float64

In [65]:
# dump lda model to disk
justlda.save('ldamodel_20_topics')

In [66]:
print("The most typical groups for every topic")
for i in range(20):
    terms = justlda.get_topic_terms(i)
    print(i, ' '.join(map(lambda x: mycorp.dictionary.get(x[0]), terms)))
    typical_groups = check_pd_20[i].sort_values(ascending=False).index[:10]
    for g in typical_groups:
        group_info = vk_get_response(
            'groups.getById', 'group_ids={0}&v=4.9&lang=ru'.format(g), access_token
        )
        print(group_info['response'][0]['name'] + ' ' + 'http://vk.com/club' + str(g))
        time.sleep(0.3)
    print()

The most typical groups for every topic
0 нeoжидaннo гepoь poбuн пpuнять oкaзaлиcь пpecтyплeние мeлoдpaмa гepou oxoт нaчaльнuкoм
Videochat/Видеочат/Webcam/Секс Гифки http://vk.com/club100491844
Эротика | Частное фото | Секс Гифки | Сиськи http://vk.com/club65098753
Перфоратор - Мужской журнал http://vk.com/club36574808
HD Кино - Фильмы онлайн 2019 http://vk.com/club123915905
НОВИНКИ КИНО 2019 http://vk.com/club90253744
HD Фильмы - Игра Престолов http://vk.com/club157344510
Audi|Ауди http://vk.com/club56513920
:D Видео Comedy Club HD http://vk.com/club190902
Весёлые картинки http://vk.com/club47813631
Interior and Decor - Дизайн интерьера. Декор http://vk.com/club37684434

1 жизнь стоить город человек вместе полный спасибо смочь ждать знать
Приколы из школы http://vk.com/club158490202
Смешные анекдоты http://vk.com/club23863253
ЧП Красноярск http://vk.com/club93025573
Твоя открыточка http://vk.com/club41495890
1000 фактов http://vk.com/club154308202
TopendBot http://vk.com/club174226458

Все-таки надо было нормально обрабатывать тексты, после данной обработки очень сложно проинтерпретировать темы.

In [69]:
justlda.show_topics(5)

[(16,
  '0.001*"8910" + 0.001*"5453" + 0.001*"9213" + 0.001*"5385" + 0.001*"5536" + 0.001*"37652" + 0.001*"22448" + 0.001*"22751" + 0.001*"26311" + 0.001*"61411"'),
 (14,
  '0.001*"5824" + 0.001*"4715" + 0.001*"6927" + 0.001*"6592" + 0.001*"6465" + 0.001*"448" + 0.001*"1226" + 0.001*"6563" + 0.001*"5750" + 0.001*"2831"'),
 (15,
  '0.000*"131216" + 0.000*"131226" + 0.000*"131231" + 0.000*"131221" + 0.000*"131232" + 0.000*"48180" + 0.000*"67267" + 0.000*"131222" + 0.000*"131223" + 0.000*"131220"'),
 (3,
  '0.000*"99530" + 0.000*"46005" + 0.000*"82648" + 0.000*"99532" + 0.000*"99531" + 0.000*"99520" + 0.000*"99533" + 0.000*"99522" + 0.000*"99537" + 0.000*"99526"'),
 (4,
  '0.000*"192629" + 0.000*"484651" + 0.000*"484650" + 0.000*"484653" + 0.000*"484652" + 0.000*"227258" + 0.000*"227283" + 0.000*"227266" + 0.000*"207795" + 0.000*"227273"')]

Судя по всему, код обработки текстов и обучения LDA модели, который нам выдали, совсем не верный. По хорошему, его надо переделать так, как нас учили на курсе по NLP: https://github.com/vlad-korotych/netology-ds4-nlp/blob/master/2.%20Морфологический%20и%20синтаксический%20анализ/hw2.ipynb
Пытаться что-нибудь показать с помощью этой модели бессмысленно. Очень плохо, когда обучающимся выдают плохой код.